<a href="https://colab.research.google.com/github/hannaofficial/GPT_using_Transformer/blob/main/basicsGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-10 08:10:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-10-10 08:10:30 (25.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt','r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(f"len of char: {len(text)}")

len of char: 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
ctoi = {ch: i for i,ch in enumerate(chars) }  #character to index
itoc = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda i: ''.join([itoc[index] for index in i])

In [41]:
print(encode("hi hanna"))
print(decode(encode("hi hanna")))

[46, 47, 1, 46, 39, 52, 52, 39]
hi hanna


In [5]:
import torch

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
data =torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
ix = torch.randint(100, (4,))
print(ix)


tensor([66, 77, 44, 34])


In [28]:
torch.manual_seed(1337)
batch_size = 64
block_size = 256

def get_batch(split):
  data = train_data if split =='train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device),y.to(device)


xb, yb = get_batch('train')
# print('inputs:')
# print(xb.shape)
# print(xb)
# print(xb)
# print(yb.shape)
# print(yb)

# print('----')

# for b in range(batch_size):
#   for t in range(block_size):
#     context = xb[b, :t+1]
#     target = yb[b, t]
#     print(f"when input is {context.tolist()} the target: {target}")



In [18]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
       X,Y = get_batch(split)
       logits, loss = model(X, Y)
       losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [19]:
max_iters = 5000
eval_interval = 500
eval_iters = 200
n_embed = 384
n_head=6
n_layer = 6
dropout = 0.2

learning_rate = 3e-4


`self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))`<br/><br/>
In PyTorch, register_buffer is a method used to register a tensor as part of a module's state, but not as a learnable parameter. This means the tensor will be saved and moved with the model (e.g., to GPU or CPU) but won't be updated during training.
<br/>**Why Use register_buffer?**<br/>
Buffers are useful for storing constant tensors or intermediate results that are essential for the model's computations but shouldn't be modified by the training process. Examples include:

 Masks: Like the tril tensor in your code, which is used to prevent the model from "looking ahead" in sequence data.

What is Broadcasting?
Broadcasting refers to the ability of PyTorch to perform arithmetic operations on tensors of different shapes by implicitly expanding them to compatible shapes without actually copying the data. This enables element-wise operations between tensors that would otherwise be incompatible.

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class Head(nn.Module):

    def __init__(self, head_size):
      super().__init__()
      self.key = nn.Linear(n_embed, head_size, bias=False)
      self.query = nn.Linear(n_embed, head_size, bias=False)
      self.value = nn.Linear(n_embed, head_size, bias=False)
      self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

      self.dropout = nn.Dropout(dropout)

    def forward(self, x):
      B,T,C = x.shape
      k = self.key(x)
      q = self.query(x)
      wei = q@k.transpose(-2, -1)*C**-0.5
      wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
      wei = F.softmax(wei, dim=-1)
      wei = self.dropout(wei)
      v = self.value(x)
      out = wei @ v
      return out

class  MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
      super().__init__()
      self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
      self.proj = nn.Linear(n_embed, n_embed)
      self.dropout = nn.Dropout(dropout)

    def forward(self,x):
      out = torch.cat([h(x) for h in self.heads], dim=-1)
      out = self.proj(out)
      out = self.dropout(out)
      return out


class FeedForward(nn.Module):
    def __init__(self, n_embed):
      super().__init__()
      self.net = nn.Sequential(nn.Linear(n_embed, 4*n_embed),
                               nn.ReLU(),
                               nn.Linear(4*n_embed, n_embed),
                               nn.Dropout(dropout))

    def forward(self, x):
       return self.net(x)

class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed//n_head
    self.sa =  MultiHeadAttention(n_head,head_size)
    self.ff = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))   #skip connection
    x = x + self.ff(self.ln2(x))
    return x


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=6) for _ in range(n_layer)])
        self.ln_final = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        idx = idx.to(device)
        tok_embed = self.token_embedding_table(idx) #BTC  batch block(time) embeddingvector size(vocab size)   here c(channel) is embed_size

        pos_embed = self.position_embedding(torch.arange(T,device=device)) # t c
        x = tok_embed + pos_embed # B T C
        x = self.blocks(x)
        x = self.ln_final(x)
        logits = self.lm_head(x)   #B T C=vocab_size   we use embed_size to reduce parameter and it's efficiency

        if targets is None:
          loss = None
        else:
          B,T,C = logits.shape
          logits = logits.view(B*T,C)  #pytorch want inout in this way
          targets = targets.view(B*T)
          loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
      idx = idx.to(device)
      for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)


      return idx
model = BigramLanguageModel(vocab_size).to(device)
xb = xb.to(device)
yb = yb.to(device)
logits,loss = model(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([16384, 65])
tensor(4.2823, device='cuda:0', grad_fn=<NllLossBackward0>)

?p-DRCUZ maQJr?F,E.HpxNDzdKHEv$fsA3WW gE,CVgX
nO,repG AQj!
,H3-fs?',ycR,SRinHPdGdNSwOkmVRMyFwLQzlLX'


In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [24]:
batch_size = 32
for steps in range(5000):
  # if iter%100 == 0:
  #   losses = estimate_loss()
  #   print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  xb,yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

1.2753785848617554


In [27]:
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=2000)[0].tolist()))


To great the father withoughts smbly may king.

KING HENRY VI:
Yes, forst thou art in true; but I need nor death
As I can putled by mew; but this be imagined
Talk that France, for aumerless a short
lawful show as those Lord Forth's life;
And thither is the time foe:
For there beares a kiss this master tender,
I would be slay like no medwell on the sequee
Of tumon woe
That that beblame bald to us.

SICINIUS:
Return,
You live,
I warranted you fair me: but, which I have not
To leaved him, sperce him to all deportunate
And therefore: no first reesolve short. what is it

First Consptivian:
I pacest being fair, washing with he hide counsell,
But did them altermity,
I'll xoured man kiss respectal of Virts,
That pretty fires with her prince father,
And she was the mighty sins of the blood,
Warwick but to the heglish'd whereinous he,
Becomen begins ere is a blow.

Gentleman:
My lord, in court weary hands!

PRINCE EDWARD:
Best, being flows thy descervice thing eye
To help the services plumant o

# **THE mathematics of Self Attention**

---



In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b,t] = torch.mean(xprev,0)

In [ ]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))  #triangular matrix
a = a/torch.sum(a, 1, keepdim=True) # we are normalizing it so that we can use it to find average
b = torch.randint(0,10,(3,2)).float()
c = a @ b #matrix multiplicatio



In [ ]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow2 = wei@x
torch.allclose(xbow[0], xbow2[0])

True

In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  #B T 16
q = query(x) #B T 16
wei = q@k.transpose(-2,-1)   # (B T 16)  @ (B 16 T) ---> (B T T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wi = F.softmax(wei, dim=-1)

v  = value(x)
out = wei@v
out.shape

torch.Size([4, 8, 16])

In [ ]:
xbow2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])